In [1]:
#Long Short Term Neural Net through Nervana's Neon package

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

In [3]:
features = pd.read_csv("~/Documents/Walmart Data/features.csv")
features.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [4]:
features.shape

(8190, 12)

In [6]:
pre_train = pd.read_csv("~/Documents/Walmart Data/train 2.csv")
pre_train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [7]:
pre_train.shape

(421570, 5)

In [8]:
stores = pd.read_csv("~/Documents/Walmart Data/stores.csv")
stores.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [9]:
stores.shape

(45, 3)

In [10]:
true_train = pd.merge(pd.merge(pre_train, features, how = 'left',
                               left_on = ['Store', 'Date'], right_on = ['Store', 'Date']), 
                      stores, left_on = ['Store'], right_on = ['Store'])
true_train['Date'] = pd.to_datetime(true_train['Date'])
true_train = true_train.drop(['IsHoliday_y', 'Type'], 1)
true_train = true_train.fillna(true_train.median())
true_train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Size
0,1,1,2010-02-05,24924.50,False,42.31,2.572,5347.45,192.0,24.6,1481.31,3359.45,211.096358,8.106,151315
1,1,1,2010-02-12,46039.49,True,38.51,2.548,5347.45,192.0,24.6,1481.31,3359.45,211.242170,8.106,151315
2,1,1,2010-02-19,41595.55,False,39.93,2.514,5347.45,192.0,24.6,1481.31,3359.45,211.289143,8.106,151315
3,1,1,2010-02-26,19403.54,False,46.63,2.561,5347.45,192.0,24.6,1481.31,3359.45,211.319643,8.106,151315
4,1,1,2010-03-05,21827.90,False,46.50,2.625,5347.45,192.0,24.6,1481.31,3359.45,211.350143,8.106,151315


In [11]:
true_train.shape

(421570, 15)

In [12]:
true_train['IsHoliday_x'] = true_train['IsHoliday_x'].astype(int) #Convert IsHolidayX to a boolean
true_train2 = true_train.sort_values('Date')
true_train2 = true_train2.set_index(true_train2['Date'])
true_train2 = true_train2.drop('Date',1)
true_train2.head()

,Store,Dept,Weekly_Sales,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Size
Date,,,,,,,,,,,,,,
2010-02-05,1,1,24924.50,0,42.31,2.572,5347.45,192.0,24.6,1481.31,3359.45,211.096358,8.106,151315
2010-02-05,29,5,15552.08,0,24.36,2.788,5347.45,192.0,24.6,1481.31,3359.45,131.527903,10.064,93638
2010-02-05,29,6,3200.22,0,24.36,2.788,5347.45,192.0,24.6,1481.31,3359.45,131.527903,10.064,93638
2010-02-05,29,7,10820.05,0,24.36,2.788,5347.45,192.0,24.6,1481.31,3359.45,131.527903,10.064,93638
2010-02-05,29,8,20055.64,0,24.36,2.788,5347.45,192.0,24.6,1481.31,3359.45,131.527903,10.064,93638


In [13]:
def timeseries_to_supervised(data, lag=1):
    df = pd.DataFrame(data)
    columns = [df.shift(i) for i in range(1, lag+1)]
    columns.append(df)
    df = pd.concat(columns, axis=1)
    df.fillna(0, inplace=True)
    return df

In [15]:
values = true_train2.values

values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(-1, 1))
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = timeseries_to_supervised(scaled, 1)

In [16]:
#reframed.drop(reframed.columns[[14,15,17,18,19,20,21,22,23,24,25,26,27]], axis=1, inplace=True)
#msk = np.random.rand(len(reframed)) < 0.985
n = 415000
train = reframed.iloc[range(n),:]
test = reframed.iloc[range(n, reframed.shape[0]),:]
train_y = scaler.inverse_transform(scaled)[range(n),2:3]
test_y = scaler.inverse_transform(scaled)[range(n, reframed.shape[0]),2:3]
train.drop(train.columns[[2,16]], axis = 1, inplace = True)
test.drop(test.columns[[2,16]], axis = 1, inplace = True)

/Library/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Library/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [17]:
test_y

array([[ 10155.18847656],
       [  2725.24316406],
       [ 27931.28710938],
       ..., 
       [  3128.16699219],
       [  5740.14941406],
       [  1076.80725098]], dtype=float32)

In [18]:
from neon.data import ArrayIterator
from neon.backends import gen_backend
gen_backend(backend='cpu', batch_size=128)

In [19]:
#make_onehot makes all our y_labels suitable for classification
train_neon = ArrayIterator(X=train, y=train_y, make_onehot=False)
test_neon = ArrayIterator(X = test, y = test_y, make_onehot = False)

In [20]:
from neon.initializers import Uniform
init_uniform = Uniform()

In [21]:
from neon.layers import LSTM, Affine, Dropout, LookupTable, RecurrentLast
from neon.transforms import Logistic, Tanh, Identity, MeanSquared
from neon.models import Model
layers = [
    LSTM(output_size=128, init=init_uniform, activation=Identity(),
         gate_activation=Identity(), reset_cells=True),
    RecurrentLast(),
    Affine(1, init_uniform, bias=init_uniform, activation=Identity())]

model = Model(layers = layers)

In [22]:
from neon.layers import GeneralizedCost
from neon.optimizers import Adagrad
from neon.callbacks import Callbacks

callbacks = Callbacks(model, eval_set=test_neon, eval_freq = 1)
cost = GeneralizedCost(costfunc=MeanSquared())
optimizer = Adagrad(learning_rate=0.01)

model.fit(train_neon,
          optimizer=optimizer,
          num_epochs=25,
          cost=cost,
          callbacks = callbacks)

Epoch 0   [Train |████████████████████| 3243/3243 batches, 252626560.00 cost, 26.37s]  [MeanSquared Loss 219535872.00, 0.20s]
Epoch 1   [Train |████████████████████| 3242/3242 batches, 246647648.00 cost, 25.82s]  [MeanSquared Loss 213379008.00, 0.13s]
Epoch 2   [Train |████████████████████| 3242/3242 batches, 244690640.00 cost, 35.10s]  [MeanSquared Loss 211511840.00, 0.13s]
Epoch 3   [Train |████████████████████| 3242/3242 batches, 240022400.00 cost, 31.17s] [MeanSquared Loss 210681408.00, 0.24s]
Epoch 4   [Train |████████████████████| 3242/3242 batches, 239766832.00 cost, 29.01s] [MeanSquared Loss 210099184.00, 0.14s]
Epoch 5   [Train |████████████████████| 3243/3243 batches, 244276640.00 cost, 32.38s]  [MeanSquared Loss 209627824.00, 0.10s]
Epoch 6   [Train |████████████████████| 3242/3242 batches, 241277440.00 cost, 29.32s]  [MeanSquared Loss 209226064.00, 0.19s]
Epoch 7   [Train |████████████████████| 3242/3242 batches, 239269664.00 cost, 31.39s]  [MeanSquared Loss 208873728.00, 0

In [23]:
from sklearn.metrics import mean_squared_error
import math

In [24]:
train_output = model.get_outputs(test_neon)
rmse = math.sqrt(mean_squared_error(train_output, test_y))
nrmse = rmse/np.mean(values)
print('Test RMSE %.3f' % rmse)
print('Test NRMSE % .3f' % nrmse)

Test RMSE 19600.835
Test NRMSE  1.649


In [25]:
train_output = model.get_outputs(train_neon)
rmse = math.sqrt(mean_squared_error(train_output, train_y))
nrmse = rmse/np.mean(values)
print('Test RMSE %.3f' % rmse)
print('Test NRMSE % .3f' % nrmse)

Test RMSE 21035.398
Test NRMSE  1.769


In [26]:
#Setting Up Data for Predictions

In [27]:
pre_test = pd.read_csv("~/Documents/Walmart Data/test 2.csv")
pre_test.head()

,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False
2,1,1,2012-11-16,False
3,1,1,2012-11-23,True
4,1,1,2012-11-30,False


In [28]:
pre_test.shape

(115064, 4)

In [29]:
true_test = pd.merge(pd.merge(pre_test, features, how = 'left',
                              left_on = ['Store', 'Date'], right_on = ['Store', 'Date']),
                     stores, left_on = ['Store'], right_on = ['Store'])
true_test['Date'] = pd.to_datetime(true_test['Date'])
true_test = true_test.drop(['IsHoliday_y', 'Type'], 1)
true_test = true_test.fillna(true_test.median())
true_test.head()

,Store,Dept,Date,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Size
0,1,1,2012-11-02,False,55.32,3.386,6766.44,5147.70,50.82,3639.90,2737.42,223.462779,6.573,151315
1,1,1,2012-11-09,False,61.24,3.314,11421.32,3370.89,40.28,4646.79,6154.16,223.481307,6.573,151315
2,1,1,2012-11-16,False,52.92,3.252,9696.28,292.10,103.78,1133.15,6612.69,223.512911,6.573,151315
3,1,1,2012-11-23,True,56.23,3.211,883.59,4.17,74910.32,209.91,303.32,223.561947,6.573,151315
4,1,1,2012-11-30,False,52.34,3.207,2460.03,742.59,3838.35,150.57,6966.34,223.610984,6.573,151315


In [30]:
true_test.shape

(115064, 14)

In [31]:
true_test['IsHoliday_x'] = true_test['IsHoliday_x'].astype(int)
true_test2 = true_test.sort_values('Date')
true_test2 = true_test2.set_index(true_test2['Date'])
true_test2 = true_test2.drop('Date',1)
true_test2.head()

,Store,Dept,IsHoliday_x,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Size
Date,,,,,,,,,,,,,
2012-11-02,1,1,0,55.32,3.386,6766.44,5147.70,50.82,3639.90,2737.42,223.462779,6.573,151315
2012-11-02,19,10,0,46.81,3.970,11686.96,18053.48,88.94,5484.50,3833.76,138.622710,7.992,203819
2012-11-02,16,55,0,38.20,3.604,2826.02,984.41,4.46,274.18,1411.99,199.290867,5.847,57197
2012-11-02,22,18,0,54.55,3.817,8793.36,10347.53,5.62,7350.89,801.32,142.661460,7.543,119557
2012-11-02,25,16,0,40.83,3.787,12435.08,16493.02,55.19,1165.36,1144.39,216.156711,7.293,128107


In [50]:
values = true_test2.values

values = values.astype('float32')
# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)
# frame as supervised learning
reframed = timeseries_to_supervised(scaled, 1)

In [51]:
fin_test = ArrayIterator(X = reframed)

In [71]:
pred = model.get_outputs(fin_test)
pred2 = pred.flat[:]
pred2

array([  262.50112915,  1338.39709473,   791.1350708 , ...,  1940.24291992,
        1980.60546875,  3502.6315918 ], dtype=float32)

In [73]:
sample = pd.read_csv("~/Documents/Walmart Data/sampleSubmission.csv")
length = np.array(sample.iloc[:,0])

array(['1_1_2012-11-02', '1_1_2012-11-09', '1_1_2012-11-16', ...,
       '45_98_2013-07-12', '45_98_2013-07-19', '45_98_2013-07-26'], dtype=object)

In [74]:
fin_df = pd.DataFrame({'Id': length, 'Weekly_Sales': pred2})

In [78]:
np.savetxt('Neon.csv', X = fin_df, delimiter = ',', header = 'Id,Weekly_Sales', comments = '', fmt = '%s')

In [56]:
reframed.shape

(115064, 26)